# Usage of the Imagine Model library

In this tutorial, we focus on introducing the model library and the IMAGINE built-in simulators as a building block of the IMAGINE package and how to use it for assembling a Bayesian analysis pipeline.
     

## 1) Initialize the models



In [36]:
import imagine as img
import imagine.observables as img_obs
import numpy as np
import astropy.units as u
from astropy.coordinates import cartesian_to_spherical

In [37]:
# Fixes numpy seed to ensure this notebook leads always to the same results
np.random.seed(42)

# A cartesian grid can be constructed as follows
cartesian_grid = img.fields.UniformGrid(box=[[-15*u.kpc, 15*u.kpc],
                                             [-15*u.kpc, 15*u.kpc],
                                             [-15*u.kpc, 15*u.kpc]],
                                        resolution = [15,15,15])


In [38]:
jf_12_field = img.fields.library.JF12Regular(grid=cartesian_grid) # using default parameters

In [39]:
jf_12_factory = img.fields.FieldFactory(
    field_class=jf_12_field,
    grid=cartesian_grid,
    active_parameters=[],
    default_parameters = {})
        

In [40]:
ymw16_field = img.fields.library.YMW16(grid=cartesian_grid) # using default parameters

In [41]:
ymw16_factory = img.fields.FieldFactory(
    field_class=ymw16_field,
    grid=cartesian_grid,
    active_parameters=[],
    default_parameters = {'central_density': 1*u.cm**-3,
                          'scale_radius': 3.0*u.kpc,
                          'scale_height': 0.5*u.kpc})

In [42]:
n_data = 500

xmax = 15
ymax = 15
zmax = 2
x = np.random.uniform(-0.9*xmax, 0.9*xmax, n_data)
y = np.random.uniform(-0.9*ymax, 0.9*ymax, n_data)
z = np.random.uniform(-0.9*zmax, 0.9*zmax, n_data)

cartesian_grid = img.fields.UniformGrid(box=[[-xmax, xmax],
                                                [-ymax, ymax],
                                                [-zmax, zmax]],
                                        resolution=[30, 30, 10])
 
dist, lat, lon = cartesian_to_spherical(x*u.kpc+8.5*u.kpc, y*u.kpc, z*u.kpc)
dist_e = np.random.normal(0, 0.1*dist)*u.kpc

fd_unit = u.rad/(u.m**2)





nside=2
size = 12*nside**2 

# Generates the container for mock dataset
fd = np.zeros(n_data) * fd_unit  # placeholder
fd_err = np.zeros(n_data) * fd_unit   # placeholder

mock_dset = {'fd': fd, 'err': fd_err, 'lat': lat, 'lon': lon}

mock_dset = img.observables.TabularDataset(mock_dset,
                                           name='fd',
                                           units=fd_unit,
                                           data_col='fd',
                                           err_col='err',
                                           lat_col='lat',
                                           lon_col='lon')

sim_config = {'lat': lat, 'lon': lon, 'dist': dist, 'e_dist': dist_e, 'grid': cartesian_grid, 'observer': np.array([-8.5, 0, 0])*u.kpc}


# Appends them to an Observables Dictionary
trigger = img_obs.Measurements(mock_dset)
print(trigger.keys())

# Prepares the Hammurabi simmulator for the mock generation
mock_generator = img.simulators.RMSimulator(measurements=trigger, sim_config=sim_config)

dict_keys([('fd', None, 'tab', None)])


TypeError: only dimensionless scalar quantities can be converted to Python scalars

In [ ]:
pipeline = img.pipelines.MultinestPipeline(run_directory = '../runs/tutorial_example/',
                                           simulator = simulator, 
                                           show_progress_reports=True,
                                           factory_list=factory_list, 
                                           likelihood=likelihood, 
                                           ensemble_size=1, n_evals_report=15)
pipeline.sampling_controllers = {'n_live_points': 500}